# Convolutional neural network

## Load X,y data from NPZ

Using the function added to *mylib.py* file, it's now easy to grab data and X/y vectors ready to be used for model training and tuning

In [32]:
# Run content of mylib.py file
%run mylib.py

# Load data from NPZ file
#data=loadNpz()
(data, X, y)=loadXy(verbose=False)

https://www.tutorialspoint.com/how-can-keras-be-used-to-extract-features-from-only-one-layer-of-the-model-using-python

In [2]:
import numpy as np


IMAGE_SIZE=data['IMAGE_SIZE']

IMAGE_COLOR_SPACE_DIM=data['IMAGE_COLOR_SPACE_DIM']

NB_LABELS=data['IMAGE_NB_LABELS']

FEATURE_SIZE=data['FEATURE_SIZE']

DATASET_NAME=data['DATASET_NAME']


KERAS_INPUT_SHAPE=(IMAGE_SIZE[0], IMAGE_SIZE[1], IMAGE_COLOR_SPACE_DIM)


X_tr=np.float32(data['trainX']['data'])
y_tr=data['trainX']['labels']

X_te=np.float32(data['test']['data'])
y_te=data['test']['labels']


In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow.keras as keras
from tensorflow.keras import activations

# Convolutional Network
model = keras.Sequential()
model.add(keras.layers.Conv2D(filters=32, kernel_size=5, strides=2,
                              activation='relu', input_shape=KERAS_INPUT_SHAPE))
model.add(keras.layers.MaxPool2D(pool_size=3))
model.add(keras.layers.Conv2D(filters=32, kernel_size=3, strides=1,
                              activation='relu'))
model.add(keras.layers.MaxPool2D(pool_size=2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(units=2048, activation='relu', name='high-level-features'))
model.add(keras.layers.Dense(units=512, activation='relu'))
model.add(keras.layers.Dense(units=512, activation='relu'))
model.add(keras.layers.Dense(units=6, activation='softmax'))

model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      2432      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 16928)             0         
_________________________________________________________________
high-level-features (Dense)  (None, 2048)              34670592  
_________________________________________________________________
dense (Dense)                (None, 512)               1049088   
__________

In [4]:
# Compile the model
model.compile(optimizer=keras.optimizers.Adamax(), loss='sparse_categorical_crossentropy', metrics=['acc'])

In [5]:
# End training when accuracy stops improving (optional)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=6)

In [6]:
# Train model

model.fit(X_tr, y_tr, epochs=100, validation_split=0.2, callbacks=[early_stopping], verbose=1, workers=-1)


Train on 336 samples, validate on 84 samples
Epoch 1/100
336/336 [==============================] - 8s 23ms/step - loss: 3.8523 - acc: 0.2530 - val_loss: 1.8643 - val_acc: 0.0000e+00
Epoch 2/100
336/336 [==============================] - 5s 13ms/step - loss: 1.6670 - acc: 0.3185 - val_loss: 1.9600 - val_acc: 0.0238
Epoch 3/100
336/336 [==============================] - 4s 13ms/step - loss: 1.5452 - acc: 0.4375 - val_loss: 2.0074 - val_acc: 0.0714
Epoch 4/100
336/336 [==============================] - 4s 13ms/step - loss: 1.3242 - acc: 0.5238 - val_loss: 1.8820 - val_acc: 0.1667
Epoch 5/100
336/336 [==============================] - 4s 13ms/step - loss: 1.1644 - acc: 0.5893 - val_loss: 1.5696 - val_acc: 0.3571
Epoch 6/100
336/336 [==============================] - 4s 13ms/step - loss: 0.8929 - acc: 0.6786 - val_loss: 1.7739 - val_acc: 0.2857
Epoch 7/100
336/336 [==============================] - 4s 13ms/step - loss: 0.6707 - acc: 0.7887 - val_loss: 1.4095 - val_acc: 0.4524
Epoch 8/100
3

In [7]:
scores = model.evaluate(X_te, y_te, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 54.90%


It's now time to save the model in order to use it in the last Notebook of this project.

As we can not use 'pickle' to save *keras.Sequential* objects, my *saveModel()* and *loadModel()* function have been improved to handle *keras.Sequential* objects.

How ?

Well, the *saveModel()* function now check the type of model passed as paramter. If it matches *keras.Sequential* type, then I use the *kera.models.save_model()* instead *pickle*.

For the *loadModel()*, as I do not know which kind of model is saved before trying to load it, I'll use a *try{} / catch Exception* block. If the files is correctly loaded using *keras.model.load_model()* then I return the *keras.Sequential* object, if not, then I use *pickle* in the *catch* block to return the model.

Here is the new saveModel() function definition:

    def saveModel(model, name):
        """
        Function that saves on disk the model passed as first parameter.
        It uses the function getModelFilename() with the 'name' parameter
        to get the filename where to save the model.
        This function checks the type of model received. If it's a keras.Sequential,
        then it uses the keras.models.save_model fuinction to save it.
        Use pickle otherwise.
        """
        filename=getModelFilename(name)
        # Save model to disk
        if isinstance(model, keras.Sequential):
            print("Saving model {} to {} using 'keras.models.save_model' library".format(name, filename))
            keras.models.save_model(model, filename, overwrite=True)
        else:
            print("Saving model {} to {} using 'pickle' library".format(name, filename))
            pickle.dump(model, open(filename, 'wb'))

And the new loadModule() function definition:

    def loadModel(name):
        """
        Function that loads from disk the model of which name is passed as first parameter.
        It uses the function getModelFilename() with the 'name' parameter
        to get the filename from where to load the model
        This function tries to first load the model using pickle. If it fails, then
        it fallback to keras.models.load_model function, as the content of the file might be
        a keras object if loading fails with pickle.
        """
        filename=getModelFilename(name)
        # load the model from disk
        print("Loading model from ", filename)
        model=None
        try:
            model=pickle.load(open(filename, 'rb'))
            print("Model loaded using pickle()")
        except:
            model=keras.models.load_model(filename)
            print("Model loaded using keras.models.load_model()")
        finally:
            return model


Now I can save my CNN model on disk, the same way I do for others.


In [33]:
saveModel(model, 'cnn')

Saving model cnn to model-cnn.sav using 'keras.models.save_model' library


In [8]:
print("Feature extraction from the model")
feature_extractor = keras.Model(
   inputs=model.inputs,
   outputs=model.get_layer(name="high-level-features").output,
)


Feature extraction from the model


In [9]:
feature_extractor.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_input (InputLayer)    (None, 299, 299, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 148, 148, 32)      2432      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 16928)             0         
_________________________________________________________________
high-level-features (Dense)  (None, 2048)              34670592  
Total para

In [10]:
print("The feature extractor method is used to get my high-level features")

features = feature_extractor.predict(X_te)
features.shape

The feature extractor method is used to get my high-level features


(51, 2048)

In [11]:
accuracy=loadModel('logistic').score(features, y_te)

Loading model from  model-logistic.sav


In [12]:
accuracy=accuracy*100
print("  {} - Accuracy on test dataset: {:.1f}%\n".format('knn', accuracy))

  knn - Accuracy on test dataset: 11.8%



In [13]:
from tensorflow.keras.models import save_model

save_model(model, 'model-cnn.sav')

In [14]:

type(model)

tensorflow.python.keras.engine.sequential.Sequential